In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import xlrd
from openpyxl import load_workbook
mapp='Alla'
files = os.listdir(mapp)
# gets all xlsx files in cwd
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
df_inf = pd.DataFrame()
for file in files_xlsx:
    data = pd.read_excel(mapp+ '/' +file, 'peptide')
    data = data.loc[(data['#Feature'] >= 4)] # sorts out if feature<4
    data['Peptide']=data['Peptide'].str.replace('[^a-zA-Z]','')
    data = data.groupby(['Peptide','Accession'], as_index=False).sum()
    df_inf = df_inf.append(data)  # appends all data

In [120]:
df_inf.loc[df_inf['Peptide'] == 'VHLTPEEKSAVTA']

,Peptide,Accession,-10lgP,Mass,Length,ppm,m/z,RT,Area Sample 13,Fraction,...,Area WF1,#Feature WF1,Area WF2,#Feature WF2,Area WF 3,#Feature WF 3,Area WF4,#Feature WF4,Area WF5,#Feature WF5
239,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,39.28,1380.7249,13,2.0,461.2498,54.34,21600000.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,48.77,1380.7249,13,2.7,691.3716,49.92,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,59.29,1380.7249,13,2.7,691.3716,54.79,NaN,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,49.17,1380.7249,13,3.6,691.3722,46.15,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,55.52,1380.7249,13,0.6,691.3701,48.11,NaN,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
817,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,34.59,1380.7249,13,0.9,461.2493,14.63,NaN,6,...,NaN,NaN,14500000.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
800,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,41.64,1380.7249,13,-0.6,461.2486,13.47,NaN,11,...,NaN,NaN,NaN,NaN,7540000.0,7.0,NaN,NaN,NaN,NaN
722,VHLTPEEKSAVTA,sp|P68871|HBB_HUMAN,47.30,1380.7249,13,1.8,691.3709,13.45,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14500000.0,7.0


In [116]:
filename='refpep.xlsx'
sheetname='Blad28'
df_ref=pd.read_excel(filename, sheet_name=sheetname)
df_ref.dropna(subset=['Peptide'], inplace=True)
df_ref

XLRDError: No sheet named <'Blad28'>

In [114]:
df_ref['Peptide']= df_ref['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
df_ref['Peptide']=df_ref['Peptide'].str.replace('[^a-zA-Z]','')
## Ta bort nedströms AA
# df_ref['Peptide'] = df_ref['Peptide'].map(lambda x: str(x[3:]) )
# df_ref['Peptide'] = df_ref['Peptide'].map(lambda x: str(x[:-3]) )
df_ref

,Unnamed: 0,Peptide,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,GEGDFLAEGGVR,NaN,NaN,NaN,NaN
1,NaN,GDSTFESKSY,NaN,NaN,NaN,https://www-sciencedirect-com.ludwig.lub.lu.se...
2,NaN,KMADEAGSEADHEGTHST,NaN,NaN,NaN,NaN
3,NaN,HWESALLR,NaN,NaN,NaN,NaN
4,NaN,IHWESASLL,NaN,NaN,NaN,NaN
5,NaN,SHALQLNNRQI,NaN,NaN,NaN,NaN
6,NaN,GGHLDQQVEEF,NaN,NaN,NaN,NaN
7,NaN,TVGSLAGQPLQERAQAWGERL,NaN,NaN,NaN,NaN


In [115]:
result=[]
for seq1 in df_ref['Peptide']:
    for seq2 in df_inf['Peptide']:
        if seq1 in seq2 and len(seq2)<(len(seq1)+6):
            result.append((seq1,seq2))
            
result=pd.DataFrame(result)
result.columns=['ref','pat']
result.drop_duplicates('pat', inplace=True)
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    result.to_excel(writer, sheetname , columns=['ref','pat'])
result

,ref,pat
0,KMADEAGSEADHEGTHST,SYKMADEAGSEADHEGTHSTKR
1,IHWESASLL,IHWESASLL
4,IHWESASLL,IHWESASLLR
7,GGHLDQQVEEF,ELGGHLDQQVEEF
8,GGHLDQQVEEF,AELGGHLDQQVEEF
9,GGHLDQQVEEF,GGHLDQQVEEF
10,GGHLDQQVEEF,LGGHLDQQVEEF
11,GGHLDQQVEEF,SLAELGGHLDQQVEEF
15,GGHLDQQVEEF,LAELGGHLDQQVEEF
